In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(model="gpt-4o",temperature=0.1)

In [4]:
def get_local_time(input: str = None) -> str:
    from datetime import datetime
    current_time = datetime.now().strftime("%H:%M:%S")
    # print(f"LocalTime tool called, returning: {current_time}")
    return f"Current local time is: {current_time}"

In [5]:
from langchain.agents import AgentExecutor, Tool

In [6]:
tools = [
    Tool(
        name="LocalTime",
        func=get_local_time,
        description="Use this tool to get the current local time."
    )
]

In [7]:
# from langchain import hub

In [8]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder,AIMessagePromptTemplate

In [9]:
system_message=SystemMessagePromptTemplate.from_template(
    """ You are an AI assistant. Your responses will be converted into speech.
        Please ensure that your answers:
        - Are conversational and sound natural when spoken.
        - Use a friendly and engaging tone.
        - Avoid overly formal or robotic language.
        - Use contractions where appropriate (e.g., "I'm" instead of "I am").
        - Keep sentences short and easy to understand.
        - Use a storytelling style when possible.
        
        Respond to the human as helpfully and accurately as possible. You have access to the following tools:
        {tools}
        Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
        Valid "action" values: "Final Answer" or {tool_names}.
        Provide only ONE action per $JSON_BLOB, as shown:
        ```
        {{
        "action": $TOOL_NAME,
        "action_input": $INPUT
        }}
        ```
        Follow this format:

        Question: input question to answer
        Thought: consider previous and subsequent steps
        Action:
        ```
        $JSON_BLOB
        ```
        Observation: action result
        ... (repeat Thought/Action/Observation N times)
        Thought: I know what to respond
        Action:
        ```
        {{
        "action": "Final Answer",
        "action_input": "Final response to human"
        }}
        ```        
        Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation
        """
)

history_placeholder=MessagesPlaceholder(variable_name="history",optional=True)

scratchpad_placeholder = MessagesPlaceholder(variable_name="agent_scratchpad", optional=True)

human_message=HumanMessagePromptTemplate.from_template(
    """{input}
    
    {agent_scratchpad}
    
    (reminder to respond in a JSON blob no matter what)"""
)

assisstant_message=AIMessagePromptTemplate.from_template(
    "Understood! I will analyze your question type and provide a detailed response."
)

In [10]:
prompt_template=ChatPromptTemplate.from_messages([
    system_message,
    history_placeholder,
    # scratchpad_placeholder,
    human_message,
    assisstant_message
])

In [11]:
prompt_template

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['history'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChu

In [12]:
from langchain.memory import ConversationBufferMemory

In [13]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

/var/folders/5b/b2jjkdb55_g_qzx3lsl5k59m0000gn/T/ipykernel_17700/1976451665.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [14]:
from langchain.agents import AgentExecutor, create_structured_chat_agent

In [15]:
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt_template)

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory,handle_parsing_errors=True, verbose=True)

In [17]:
from langchain.schema import AIMessage

In [18]:
response = agent_executor.invoke({
    "input": "What is 15^2?",
    "history": memory.chat_memory.messages,  
    # "agent_scratchpad": ''
})

print(response)



> Entering new AgentExecutor chain...
Thought: I need to calculate 15 squared, which is a straightforward math problem.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "15 squared is 225."
}
```

> Finished chain.
{'input': 'What is 15^2?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={})], 'output': '15 squared is 225.'}


In [19]:
response['output']

'15 squared is 225.'

In [20]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={})]), return_messages=True)

In [21]:
response = agent_executor.invoke({
    "input": "how are you?",
})

print(response)



> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Final Answer",
  "action_input": "I'm doing great, thanks for asking! How about you?"
}
```

> Finished chain.
{'input': 'how are you?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={})], 'output': "I'm doing great, thanks for asking! How about you?"}


In [22]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={})]), return_messages=True)

In [23]:
response = agent_executor.invoke({
    "input": "what is the time now?",
})

print(response)



> Entering new AgentExecutor chain...
Thought: I need to find out the current local time to answer the question.
Action:
```json
{
  "action": "LocalTime",
  "action_input": ""
}
```Current local time is: 22:39:35Action:
```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 10:39 PM."
}
```

> Finished chain.
{'input': 'what is the time now?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={})], 'output': 'The current local time is 10:39 PM.'}


In [24]:
memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={})]), return_messages=True)

In [25]:
memory.chat_memory.messages

[HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={})]

In [26]:
response = agent_executor.invoke({
    "input": "Do you know what my last question was?",
})

print(response)



> Entering new AgentExecutor chain...
```json
{
  "action": "Final Answer",
  "action_input": "Your last question was about the current time."
}
```

> Finished chain.
{'input': 'Do you know what my last question was?', 'history': [HumanMessage(content='What is 15^2?', additional_kwargs={}, response_metadata={}), AIMessage(content='15 squared is 225.', additional_kwargs={}, response_metadata={}), HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing great, thanks for asking! How about you?", additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Do you know what my last question was?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your last question was about the current time.', additional_kwargs={}, response

In [27]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

In [28]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    memory=memory,
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [29]:
response = agent_executor.invoke({
    "input": "what is the time now?",
})

print(response)



> Entering new AgentExecutor chain...
Thought: I need to find out the current local time to answer the question accurately.
Action:
```json
{
  "action": "LocalTime",
  "action_input": ""
}
```Current local time is: 22:39:38Action:
```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 10:39 PM."
}
```

> Finished chain.
{'input': 'what is the time now?', 'history': [HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={})], 'output': 'The current local time is 10:39 PM.'}


In [30]:
agent_executor.memory.buffer

[HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={})]

In [31]:
memory = ConversationBufferMemory(
    memory_key="history", return_messages=True)

In [32]:
class DebugAgentExecutor(AgentExecutor):
    def _call(self, inputs, run_manager=None):
        response = super()._call(inputs, run_manager)
        print("test test test")
        return response

agent_executor = DebugAgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)


In [33]:
response = agent_executor.invoke({
    "input": "what is the time now?",
})

print(response)



> Entering new DebugAgentExecutor chain...
Thought: I need to find out the current local time to answer the question accurately.
Action:
```json
{
  "action": "LocalTime",
  "action_input": ""
}
```Current local time is: 22:39:40Action:
```json
{
  "action": "Final Answer",
  "action_input": "The current local time is 10:39 PM."
}
```
test test test

> Finished chain.
{'input': 'what is the time now?', 'history': [HumanMessage(content='what is the time now?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current local time is 10:39 PM.', additional_kwargs={}, response_metadata={})], 'output': 'The current local time is 10:39 PM.'}


In [34]:
response['output']

'The current local time is 10:39 PM.'

In [35]:
# from langchain.agents import (
#     AgentExecutor,
#     create_react_agent,
# )

In [36]:
# agent = create_react_agent(
#     llm=llm,
#     tools=tools,
#     prompt=prompt,
#     stop_sequence=True,
# )

In [37]:
# agent_executor = AgentExecutor.from_agent_and_tools(
#     agent=agent,
#     tools=tools,
#     verbose=True,
#     memory=memory,
#     handle_parsing_errors=True
# )

In [38]:
# response = agent_executor.invoke({"input": "What time is it?"})